In [1]:
from data.data import COVID19India

In [2]:
cin = COVID19India()

In [3]:
df = cin.moh_data(save=False)

In [4]:
df.tail()

,Name of State / UT,Total Confirmed cases (Indian National),Total Confirmed cases ( Foreign National ),Cured/Discharged/Migrated,Death
22,Ladakh,13,0,0,0
23,Uttar Pradesh,36,1,11,0
24,Uttarakhand,4,1,0,0
25,West Bengal,9,0,0,1
26,Total number of confirmed cases in India,602,47,43,13


In [10]:
import pandas as pd

In [11]:
from pathlib import Path

In [12]:
import pandas as pd
import numpy as np
import requests

markdown_file = open('help.md')
file_content = markdown_file.read()
md = markdown.Markdown(extensions=['tables', 'codehilite', 'fenced_code'])
pg = md.convert(file_content)